### Create Digital Elevation Model using GRASS GIS

In [ ]:
alti = '/mnt/x/pwgis/code24data/demo1grass/countours.shp'
cellsize = 10
col = 'ELEVATION'

dem = '/mnt/x/pwgis/code24data/results/demo1grass.tif'

In [ ]:
import os
from osgeo import ogr, gdal
import numpy as np
from glass.pys.oss import mkdir
from glass.wenv.grs import run_grass

In [ ]:
"""
GDAL Drivers Name
"""

def drv_name(_file):
    """
    Return the driver for a given file format
    """
    
    drv = {
        # Vector files
        '.gml'    : 'GML',
        '.shp'    : 'ESRI Shapefile',
        '.json'   : 'GeoJSON',
        '.kml'    : 'KML',
        '.osm'    : 'OSM',
        '.dbf'    : 'ESRI Shapefile',
        '.vct'    : 'Idrisi',
        '.nc'     : 'netCDF',
        '.vrt'    : 'VRT',
        '.mem'    : 'MEMORY',
        '.sqlite' : 'SQLite',
        '.gdb'    : 'FileGDB',
        # Raster files
        '.tif'    : 'GTiff',
        '.ecw'    : 'ECW',
        '.mpr'    : 'ILWIS',
        '.mpl'    : 'ILWIS',
        '.jpg'    : 'JPEG',
        '.nc'     : 'netCDF',
        '.png'    : 'PNG',
        '.vrt'    : 'VRT',
        '.asc'    : 'AAIGrid',
        '.img'    : 'HFA',
        # Vector or Raster
        '.gpkg'   : 'GPKG'
    }
    
    return str(drv[os.path.splitext(_file)[1]])

In [ ]:
def shpext_to_rst(shp, cellsize, outrst):
    """
    Create a template raster using ESRI Shapefile
    Extent
    """
    
    # Get shapefile extent
    dt = ogr.GetDriverByName(drv_name(shp)).Open(shp, 0)
    lyr = dt.GetLayer()
    ext = list(lyr.GetExtent())
    left, right, bottom, top = ext
    
    spref = lyr.GetSpatialRef()
    
    dt.Destroy()
    
    rows = (float(top) - float(bottom)) / cellsize
    cols = (float(right) - float(left)) / cellsize
    
    rows = int(rows) if rows == int(rows) else int(rows) + 1
    cols = int(cols) if cols == int(cols) else int(cols) + 1
    
    nrst = np.full((rows, cols), 1)
    
    # Create new raster
    img = gdal.GetDriverByName(drv_name(outrst)).Create(
        outrst, cols, rows, 1, gdal.GDT_Byte
    )
    
    img.SetGeoTransform((left, cellsize, 0, top, 0, -cellsize))
    
    band = img.GetRasterBand(1)
    
    band.WriteArray(nrst)
    
    img.SetProjection(spref.ExportToWkt())
    
    band.FlushCache()
    
    return outrst

In [ ]:
def shp_to_grs(inLyr, outLyr):
    """
    Add Shape to GRASS GIS
    """
    
    from grass.pygrass.modules import Module
        
    m = Module(
        "v.in.ogr", input=inLyr, output=outLyr, flags='o',
        overwrite=True, run_=False, quiet=True
    )
        
    m()
    
    return outLyr

In [ ]:
def ridw(inRst, outRst, numberPoints=None):
    """
    r.surf.idw - Provides surface interpolation from raster point data
    by Inverse Distance Squared Weighting.
    
    r.surf.idw fills a grid cell (raster) matrix with interpolated values
    generated from input raster data points. It uses a numerical approximation
    technique based on distance squared weighting of the values of nearest data
    points. The number of nearest data points used to determined the
    interpolated value of a cell can be specified by the user (default:
    12 nearest data points).
    
    If there is a current working mask, it applies to the output raster map.
    Only those cells falling within the mask will be assigned interpolated
    values. The search procedure for the selection of nearest neighboring
    points will consider all input data, without regard to the mask.
    The -e flag is the error analysis option that interpolates values
    only for those cells of the input raster map which have non-zero
    values and outputs the difference (see NOTES below).
    
    The npoints parameter defines the number of nearest data points used to
    determine the interpolated value of an output raster cell.
    """
    
    from grass.pygrass.modules import Module
    
    numberPoints = 12 if not numberPoints else numberPoints
    
    idw = Module(
        'r.surf.idw', input=inRst, output=outRst, npoints=numberPoints,
        run_=False, quiet=True, overwrite=True
    )
    
    idw()

In [ ]:
def rstcalc(expression, output):
    """
    Basic Raster Calculator
    """
    
    from grass.pygrass.modules import Module
        
    rc = Module(
        'r.mapcalc',
        f'{output} = {expression}',
        overwrite=True, run_=False, quiet=True
    )
        
    rc()
    
    return output

In [ ]:
def grsshp_to_grsrst(inshp, src, outrst, cmd=None):
    """
    GRASS Vector to GRASS Raster

    api:
    * pygrass
    * grass

    Vectorial geometry to raster
    
    If source is None, the convertion will be based on the cat field.
    
    If source is a string, the convertion will be based on the field
    with a name equal to the given string.
    
    If source is a numeric value, all cells of the output raster will have
    that same value.
    """

    __USE = "cat" if not src else "attr" if type(src) == str \
        else "val" if type(src) == int or \
        type(src) == float else None
    
    if not __USE:
        raise ValueError('\'source\' parameter value is not valid')
    
    from grass.pygrass.modules import Module
            
    m = Module(
        "v.to.rast", input=inshp, output=outrst, use=__USE,
        attribute_column=src if __USE == "attr" else None,
        value=src if __USE == "val" else None,
        overwrite=True, run_=False, quiet=True
    )
            
    m()

    return outrst

In [ ]:
def grs_to_rst(grsRst, rst):
    """
    GRASS Raster to Raster
    """
    
    grass_formats = {
        '.tif': 'GTiff',
        '.img': 'HFA'
    }
    
    fn, ff = os.path.splitext(rst)
    
    from grass.pygrass.modules import Module
        
    m = Module(
        "r.out.gdal", input=grsRst, output=rst,
        format=grass_formats[ff], flags='c',
        createopt='TFW=YES',
        overwrite=True, run_=False, quiet=True
    )
        
    m()
    
    return rst

In [ ]:
# Create tempfolder
workfolder = mkdir(os.path.dirname(dem), randName=True, overwrite=True)

# Generate template raster
refrst = shpext_to_rst(alti, cellsize, os.path.join(workfolder, 'refraster.tif'))

In [ ]:
# Start GRASS GIS session
loc = 'loc_dem'
gb = run_grass(workfolder, location=loc, srs=refrst)

import grass.script.setup as gsetup

gsetup.init(gb, workfolder, loc, 'PERMANENT')

In [ ]:
# Import elevation data
elv = shp_to_grs(alti, 'elevation')

# Elevation (GRASS Vector) to Raster
elevRst = grsshp_to_grsrst(elv, col, 'rst_elevation')

# Multiply cells values by 100 000.0
rstcalc('int(rst_elevation * 100000)', 'rst_elev_int')

# Run IDW to generate the new DEM
ridw('rst_elev_int', 'dem_int', numberPoints=15)

# DEM to Float
rstcalc('dem_int / 100000.0', 'fdem')

grs_to_rst('fdem', dem)